# image_classification_preprocess.py

In [ ]:
# from image_classification_preprocess_sub import exec_process
# import logging

# def process_for_train(pm):
#     exec_process(pm)
#     logging.info('[hunmin log] the end line of the function [process_for_train]')


# def init_svc(im, rule):
#     return {}


# def transform(df, params, batch_id):
#     logging.info('[hunmin log] df.shape : {}'.format(df.shape))
#     logging.info('[hunmin log] type(df) : {}'.format(type(df)))
#     logging.info('[hunmin log] the end line of the function [transform]')
#     return df

# image_classification_preprocess_sub.py

In [ ]:
# import os
# import zipfile
# import logging

# def exec_process(pm):

#     logging.info('[hunmin log]  the start line of the function [exec_process]')

#     # 저장 파일 확인
#     list_files_directories(pm.source_path)

#     # pm.source_path의 dataset.zip 파일을
#     # pm.target_path 경로에 압축해제
#     my_zip_path = os.path.join(pm.source_path,'dataset.zip')
#     extract_zip_file = zipfile.ZipFile(my_zip_path)
#     extract_zip_file.extractall(pm.target_path)
#     extract_zip_file.close()

#     # 저장 파일 확인
#     list_files_directories(pm.target_path)

#     logging.info('[hunmin log]  the finish line of the function [exec_process]')


# # 저장 파일 확인
# def list_files_directories(path):
#     # Get the list of all files and directories in current working directory
#     dir_list = os.listdir(path)
#     logging.info('[hunmin log] Files and directories in {} :'.format(path))
#     logging.info('[hunmin log] dir_list : {}'.format(dir_list))




# train.py

In [ ]:
import logging
from train_sub import exec_train
import t3qai_client as tc
from t3qai_client import T3QAI_TRAIN_OUTPUT_PATH, T3QAI_TRAIN_MODEL_PATH, T3QAI_TRAIN_DATA_PATH, T3QAI_TEST_DATA_PATH, T3QAI_MODULE_PATH

def main():
    result = None
    result_msg = "success"
    a = tc.t3qai_client()
    a.train_start()
    try:
        train()
    except Exception as e:
        result = e
        result_msg = e
        logging.info('error log : {}'.format(e))
    a.train_finish(result, result_msg)

def train():
    exec_train()
    logging.info('[hunmin log] the end line of the function [train]')

if __name__ == '__main__':
    main()

# train_sub.py

In [ ]:
import t3qai_client as tc
from t3qai_client import T3QAI_TRAIN_OUTPUT_PATH, T3QAI_TRAIN_MODEL_PATH, T3QAI_TRAIN_DATA_PATH, T3QAI_TEST_DATA_PATH, T3QAI_MODULE_PATH

# Imports
import torch
import torchvision
import os
import logging
import matplotlib.pyplot as plt
from ultralytics import YOLO
import psutil

# 사용할 gpu 번호를 적는다.
# os.environ["CUDA_VISIBLE_DEVICES"]='0'
# 환경 변수 설정 (OpenMP 문제 해결)
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'


def exec_train():
  logging.info('[hunmin log] the start line of the function [exec_train]')
  logging.info('[hunmin log] T3QAI_TRAIN_DATA_PATH : {}'.format(T3QAI_TRAIN_DATA_PATH))
  
  # 저장 파일 확인
  list_files_directories(T3QAI_TRAIN_DATA_PATH)
  
  yaml_path = f"{T3QAI_TRAIN_DATA_PATH}/dataset.yaml"

  dataset_yaml = f"""
    train: '{T3QAI_TRAIN_DATA_PATH}/dataset/train/images'
    val: '{T3QAI_TRAIN_DATA_PATH}/dataset/val/images'

    names: ['Road_No_Parking', 'Road_Speed_Limit_in_School_Zone', 'Road_School_Zone', 'Crosswalk',  'Road_No_Stopping_or_Parking',  'Road_No_Stopping_Zone', 'stop', 'traffic_lane_yellow_solid',  'school_zone',  'no_parking',  'fire_hydrant']

    train_annotation: '{T3QAI_TRAIN_DATA_PATH}/dataset/train/labels'
    val_annotation: '{T3QAI_TRAIN_DATA_PATH}/dataset/val/labels'
  """

  with open(yaml_path, 'w') as file:
      file.write(dataset_yaml)
  
  
  ### 데이터 전처리와 학습 모델을 구성하고 모델 학습을 수행
  model = YOLO("yolov8n.pt")
  # model = YOLO(r"C:\Users\LMK\Desktop\output\best.pt")

  # 시스템 메모리와 CPU 사용량 모니터링
  print(f"CPU usage: {psutil.cpu_percent()}%")
  print(f"Memory usage: {psutil.virtual_memory().percent}%")

  # 학습 시작
  model.train(
      data=yaml_path,
      epochs=100,
      imgsz=640,
      batch=32,        # 배치 크기 줄이기
      workers=6,      # 워커 수 줄이기
      name='yolov8_multiclass',
      cache=True,     # 캐시 사용
      project= os.path.join(T3QAI_TRAIN_OUTPUT_PATH, 'logs'), # '.\\multi_V8\\logs',
      exist_ok=False
  ) 
  

  ### 학습 모델의 성능을 검증하고 배포할 학습 모델을 저장
  ### 전처리 객체와 학습 모델 객체를 T3QAI_TRAIN_MODEL_PATH 에 저장
  model.export(format="onnx")  # creates 'yolov8n.onnx'

  ### 학습 결과를 파일(이미지, 텍스트 등) 형태로 T3QAI_TRAIN_OUTPUT_PATH 에 저장 
  
  # 모델 로드
  model = YOLO(f"{T3QAI_TRAIN_DATA_PATH}/logs/yolov8_multiclass3/weights/best.pt")

  # 성능 평가 수행
  results = model.val(data=yaml_path, conf=0.25, iou=0.5)

  # 평가 결과 출력
  print("Evaluation Results:")
  print(f"Precision: {results.metrics.precision:.4f}")
  print(f"Recall: {results.metrics.recall:.4f}")
  print(f"mAP@0.5: {results.metrics.map50:.4f}")
  print(f"mAP@0.5:0.95: {results.metrics.map:.4f}")

  # 결과를 파일로 저장
  #os.makedirs(T3QAI_TRAIN_OUTPUT_PATH, exist_ok=True)
  with open(os.path.join(T3QAI_TRAIN_OUTPUT_PATH, 'evaluation_results.txt'), 'w') as f:
      f.write("Evaluation Results:")
      f.write(f"Precision: {results.metrics.precision:.4f}\n")
      f.write(f"Recall: {results.metrics.recall:.4f}\n")
      f.write(f"mAP@0.5: {results.metrics.map50:.4f}\n")
      f.write(f"mAP@0.5:0.95: {results.metrics.map:.4f}\n")


  # 모델 평가 (Evaluate Model) 및 시각화




###########################################################################
## exec_train() 호출 함수 
###########################################################################
      
# 저장 파일 확인
def list_files_directories(path):
    # Get the list of all files and directories in current working directory
    dir_list = os.listdir(path)
    logging.info('[hunmin log] Files and directories in {} :'.format(path))
    logging.info('[hunmin log] dir_list : {}'.format(dir_list)) 

# inference_service.py

In [ ]:
import t3qai_client as tc
from t3qai_client import T3QAI_INIT_MODEL_PATH
from inference_service_sub import exec_inference_file # ,exec_init_model

import logging
logger = logging.getLogger()
logger.setLevel('INFO')

# def init_model():
#   ''' 
#     - 전처리 객체 불러오기
#     - 학습모델 객체 불러오기
#       에 필요한 코드 작성 
#   '''
#   params = exec_init_model()
#   logging.info('[hunmin log] the end line of the function [init_model]')
#   return { **params }


def inference_file(files):
  '''
    - files 입력에 대한 추론 처리 기능
    - 추론시 입력 데이터에 대한 전처리(Data Preprocessing)
    - 추론(Inference) 또는 예측(Prediction) 
    - 추론 결과 데이터 후처리(Data Postprocessing) 
  '''
  result = exec_inference_file(files)
  logging.info('[hunmin log] the end line of the function [inference]')
  return result


# # inference의 return을 DownloadFile 으로 할때 실행합니다.
# inference_result = inference_file(["C:/Users/tjral/Desktop/OD/meta_data/dataset/val/images/891314_614.jpg"])
# print(inference_result)

# inference_service_sub.py

In [ ]:
import logging, os
from ultralytics import YOLO
from t3qai_client import T3QAI_INIT_MODEL_PATH, T3QAI_TRAIN_MODEL_PATH

# def exec_init_model():
#   '''init_model에서 사용''''
#   model_path = os.path.join(T3QAI_INIT_MODEL_PATH, 'yolov8_multiclass.pt')
#   model = YOLO(model_path)
#   # model_info_dict = {
#   #     "model": model
#   # }
#   return model

def extract_labels(results, model_names):
      detected_objects = []
      for box in results[0].boxes:
          cls = int(box.cls[0].item())  # 클래스 번호 추출
          conf = box.conf[0].item()  # 신뢰도 값 추출
          detected_objects.append((model_names[cls], conf))
      return detected_objects

def exec_inference_file(files):
    """
    inference_file에서 사용
    파일기반 추론함수는 files와 로드한 model을 전달받습니다.
    """
    logging.info('[hunmin log] the start line of the function [exec_inference_file]')

    # model_path = os.path.join(T3QAI_INIT_MODEL_PATH, 'best.pt')
    
    model_path = f"{T3QAI_TRAIN_MODEL_PATH}/logs/yolov8_multiclass/weights/best.pt"
    model = YOLO(model_path)

    result = {}
    
    for idx, one_file in enumerate(files):
        logging.info('[hunmin log] file inference')
        
        inference_file = one_file.file
        # image = Image.open(inference_file).convert('L')
        # image = image.resize((28, 28))
        # image = np.invert(image).astype('float32')/255.
        # image = image.reshape(-1, 28, 28 , 1)

        logging.info('[hunmin log] load model')
        # data predict
        
        output = model(inference_file)
        
        if len(output) == 0:
          return None, [], None, "Nothing detected in OD"

        print(model.names)
        od_result = extract_labels(output, model.names)
        od_result = [label for label, _ in od_result]
        
        result[f'file{idx}_inference'] = od_result
#     result = [DownloadFile(file_path=T3QAI_TRAIN_OUTPUT_PATH+'/Accuracy_Loss.png', file_name='result.jpg'), 
#               DownloadFile(file_path=T3QAI_TRAIN_OUTPUT_PATH+'/Accuracy_Loss.png', file_name='result2.jpg')]
    return result



# class

In [ ]:
import os
import shutil
import tempfile
import base64
import pandas as pd

import ipywidgets
from ipywidgets import FileUpload
from IPython.display import FileLink

# t3qai_client 클래스: t3qai_client 객체
class t3qai_client:
    def train_start(self):
        return None

    def train_finish(self, result, result_msg):
        if result_msg != "success":
            raise Exception(result_msg)
        else:
            logging.info(result)
            logging.info("train finish")

    def train_load_param(self):
        '''set_param'''
        epoch = 20
        batch_size = 16
        params = {"epoch" : epoch, 'batch_size' : batch_size}
        return { **params }

class PM:
    def __init__(self):
        self.source_path = './'
        self.target_path = './meta_data'

class UploadFile:
    def __init__(self, file, filename):
        self.file = file
        self.filename = filename

def DownloadFile(file_name, file_obj = None, file_path = None):
    file_route = './meta_data/DownloadFiles'
    os.makedirs(file_route, exist_ok = True)
    file_dir = os.path.join(file_route, file_name)
    if (file_obj == None) == (file_path == None):
        Err_msg = "[DownloadFile Error]: Only one of the 'file_path' or 'file_obj' arguments is required."
        Err_msg += f"{0 if file_obj==None else 2} arguments entered."
        raise Exception(Err_msg)
    elif(file_obj != None):
        file_obj.seek(0)
        file_read = base64.b64encode(file_obj.read()).decode('utf-8')
        binary_file = base64.b64decode(file_read)
        with open(file_dir, 'wb') as f:
            f.write(binary_file)
    elif(file_path != None):
        shutil.copyfile(file_path, file_dir)

    return FileLink(file_dir)

pm = PM()

T3QAI_TRAIN_OUTPUT_PATH = './meta_data'
T3QAI_TRAIN_MODEL_PATH = './meta_data'
T3QAI_TRAIN_DATA_PATH = './meta_data'
T3QAI_TEST_DATA_PATH = './meta_data'
T3QAI_MODULE_PATH = './meta_data'
T3QAI_INIT_MODEL_PATH = './meta_data'


# t3qai_client 객체
tc = t3qai_client()
print('T3QAI_TRAIN_OUTPUT_PATH:', T3QAI_TRAIN_OUTPUT_PATH)
print('T3QAI_TRAIN_MODEL_PATH:', T3QAI_TRAIN_MODEL_PATH)
print('T3QAI_TRAIN_DATA_PATH:', T3QAI_TRAIN_DATA_PATH)
print('T3QAI_TEST_DATA_PATH:', T3QAI_TEST_DATA_PATH)
print('T3QAI_MODULE_PATH:', T3QAI_MODULE_PATH)
print('T3QAI_INIT_MODEL_PATH:', T3QAI_INIT_MODEL_PATH)

# init_svc(im, rule) 함수 입력
im = None
rule = None
# transform(df, params, batch_id) 함수 입력
batch_id = 0


import io
import pandas as pd

# base64 encoded image - apple.jpg
data = [['iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAIAAAD9b0jDAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAAACySURBVEhL7ZLRDoAgCEWt//9nc8EIEepStvXQeWkyPEKw1FrLbFb+TuWXzichXXb4cAokJR0tH+JFK21G8V6CSqlApMxGelBIsVBHeOOEzZYGdRzpussfL7eIazHPa0wrx0GMdBSiuMbkdINyb5og0gRL3dTbcPtNOpYZveQ2pA1n0hTakF5+hA9Lzd87pNFYLhkvsvT2lMhorndluxkRUuCYbzcp9ROi55+up8sLK1XKBj1wbx3DelAOAAAAAElFTkSuQmCC']]
df = pd.DataFrame(data)
print('df: ', df)
print('df.dtypes:', df.dtypes)

# inference_file 함수 추론
files = []

uploader = FileUpload(accept='*', multiple=True, description='select data', button_style='danger')
def uploader_change(change):
    uploader.button_style='success'
    count = len(uploader.value)
    uploader._counter = count
    files.clear()
    for file_num in range(count):
        temp_data = tempfile.TemporaryFile()
        if ipywidgets.__version__[0] == '7':
            temp_data.write(list(uploader.value.values())[file_num]['content'])
            file = UploadFile(temp_data, pd.DataFrame(list(uploader.value.values())[file_num]).iloc[1,0])
        elif int(ipywidgets.__version__[0]) > 7:
            temp_data.write(uploader.value[file_num].content)
            file = UploadFile(temp_data, uploader.value[file_num].name)
        files.append(file)

uploader.observe(uploader_change, 'value')

In [ ]:
# %%time
# process_for_train(pm)

In [ ]:
%%time
# main() 함수에서 train() 함수 실행
main()

In [ ]:
# uploader widget(해당 커널 output의 버튼)에 파일을 업로드 한 뒤 infernece_file으로 추론합니다.
display(uploader)

In [ ]:
# DataFrame 입력에 대한 추론 결과를 딕셔너리(Dictionary) 형태로 리턴(return)
%%time
inference_file(files)

In [ ]:
%%time
# inference의 return을 DownloadFile 으로 할때 실행합니다.
inference_result = inference_file(files)

if type(inference_result) == list:
    display(*inference_result)
else:
    display(inference_result)